<h1>Purpose </h1>
This notebook serves as a submission for the IBM professional data science certificate capstone project "Battle of the neighborhoods"


# Begin with imports and data acquisition

In [30]:
import pandas as pd
import numpy as np
data_source = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

data.head(15)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


## Clean data to much criteria:
* Only process cells that have an assigned borough, ignore others.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.
* If the neighborhood is unassigned, assign it to be the same as the Borough


#### Begin by removing missing borough values

In [31]:
#Remove values where borough is unassigned
#Begin by replacing not assigned with NaT
data['Borough'].replace("Not assigned",pd.NaT,regex=True,inplace=True)
#Now drop NaT values in Borough
data.dropna(subset = ['Borough'],inplace=True)
data.reset_index(inplace=True)
data

,index,Postal code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Regent Park / Harbourfront
3,5,M6A,North York,Lawrence Manor / Lawrence Heights
4,6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...,...
98,160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,Business reply mail Processing CentrE
101,169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


#### Handle missing neighborhoods

In [32]:
data['Neighborhood'].replace(np.NaN,data.Neighborhood,regex=True,inplace=True)

data = data[['Postal code' , 'Borough' ,'Neighborhood']]
data

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


#### Group by postal code and seperate by coloumns

In [33]:
#Remove space from Postal code
data.columns
column_names=['Postal_code', 'Borough', 'Neighborhood']
data.columns=column_names
data.columns


Index(['Postal_code', 'Borough', 'Neighborhood'], dtype='object')

In [34]:
counts = data.Postal_code.value_counts()
counts[counts>1]
data = data.groupby('Borough').apply(lambda x: x.apply(lambda y:  ', '.join(y) \
                                                   if y.name =='Neighbourhood' \
                                                   else y.tolist()[0]))
data.reset_index(inplace=True, drop=True)
data

,Postal_code,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M5A,Downtown Toronto,Regent Park / Harbourfront
2,M4E,East Toronto,The Beaches
3,M4B,East York,Parkview Hill / Woodbine Gardens
4,M9A,Etobicoke,Islington Avenue
5,M7R,Mississauga,Canada Post Gateway Processing Centre
6,M3A,North York,Parkwoods
7,M1B,Scarborough,Malvern / Rouge
8,M6H,West Toronto,Dufferin / Dovercourt Village
9,M6C,York,Humewood-Cedarvale


In [35]:
#Replace / with ,
data['Neighborhood'] = data['Neighborhood'].str.replace('/',',')
data

,Postal_code,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M5A,Downtown Toronto,"Regent Park , Harbourfront"
2,M4E,East Toronto,The Beaches
3,M4B,East York,"Parkview Hill , Woodbine Gardens"
4,M9A,Etobicoke,Islington Avenue
5,M7R,Mississauga,Canada Post Gateway Processing Centre
6,M3A,North York,Parkwoods
7,M1B,Scarborough,"Malvern , Rouge"
8,M6H,West Toronto,"Dufferin , Dovercourt Village"
9,M6C,York,Humewood-Cedarvale


## Next step is to add a lattitude and longitude value for each Postal code

### Data has been downloaded to csv file for convenience and reliability

In [36]:
#First load the data into a dictionary of the following format:
# {postal_code:(lat,long)}
import csv
GeoDict={}
reader = csv.DictReader(open('Geospatial_Coordinates.csv'))
for row in reader:

    code=row["Postal Code"]
    latlong=(row["Latitude"],row["Longitude"])
    GeoDict.update({code:latlong})
    


### Then the coordinates are prepared to be added to the DF

#### This is done by iterating over the postal codes in the DF and looking up their values in a dictionary

In [49]:
#First expand the data frame
data["Latitude"] = ""
data["Longitude"]= ""


In [51]:
for index,row in data.iterrows():
    row["Latitude"],row["Longitude"]=GeoDict[row["Postal_code"]]

In [52]:
data.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6542599,-79.3606359
2,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
3,M4B,East York,"Parkview Hill , Woodbine Gardens",43.7063972,-79.309937
4,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
